In [89]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [90]:
# import Health and Physical Activity data
health = pd.read_excel('data/FoodAvailabilityDataset.xls',sheet_name='HEALTH')
health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FIPS                   3143 non-null   int64  
 1   State                  3143 non-null   object 
 2   County                 3143 non-null   object 
 3   PCT_DIABETES_ADULTS08  3138 non-null   float64
 4   PCT_DIABETES_ADULTS13  3142 non-null   float64
 5   PCT_OBESE_ADULTS12     3143 non-null   float64
 6   PCT_OBESE_ADULTS17     3143 non-null   float64
 7   PCT_HSPA17             2383 non-null   float64
 8   RECFAC11               3143 non-null   int64  
 9   RECFAC16               3143 non-null   int64  
 10  PCH_RECFAC_11_16       3000 non-null   float64
 11  RECFACPTH11            3143 non-null   float64
 12  RECFACPTH16            3143 non-null   float64
 13  PCH_RECFACPTH_11_16    3000 non-null   float64
 14  NATAMEN                3108 non-null   float64
dtypes: f

In [91]:
# get variable that we'll be solving for: "Adult obesity rate by county in 2017" [PCT_OBESE_ADULTS17]
obesity=health[['FIPS','State','County','PCT_OBESE_ADULTS17']]
obesity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   FIPS                3143 non-null   int64  
 1   State               3143 non-null   object 
 2   County              3143 non-null   object 
 3   PCT_OBESE_ADULTS17  3143 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 98.3+ KB


In [92]:
# get health variables 
# future data (after 2010 will NOT be included)
# obesity rates for previous years and diabetes rates will NOT be included
# [PCT_HSPA09] feature has too many NULLs
# health=health[['FIPS','State','County','RECFACPTH07','NATAMEN']]
# health.drop('FIPS',axis=1).corr()
health = health[['FIPS','RECFACPTH16', 'NATAMEN']].dropna()
health.corr()

,FIPS,RECFACPTH16,NATAMEN
FIPS,1.000000,0.034811,-0.081527
RECFACPTH16,0.034811,1.000000,0.038860
NATAMEN,-0.081527,0.038860,1.000000


In [93]:
# import Access and Proximity to Grocery Store data 
access = pd.read_excel('data/FoodAvailabilityDataset.xls',sheet_name='ACCESS')
access.describe()

,FIPS,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,PCH_LACCESS_LOWI_10_15,PCT_LACCESS_LOWI10,...,LACCESS_HISP15,PCT_LACCESS_HISP15,LACCESS_NHASIAN15,PCT_LACCESS_NHASIAN15,LACCESS_NHNA15,PCT_LACCESS_NHNA15,LACCESS_NHPI15,PCT_LACCESS_NHPI15,LACCESS_MULTIR15,PCT_LACCESS_MULTIR15
count,3143.000000,3143.000000,3124.000000,3.117000e+03,3143.000000,3124.000000,3143.000000,3123.000000,3.115000e+03,3143.000000,...,3124.000000,3124.000000,3124.000000,3124.000000,3124.000000,3124.000000,3124.000000,3124.000000,3124.000000,3124.000000
mean,30390.411709,20156.653242,19287.747386,7.180008e+03,23.559330,23.084470,5551.776559,5833.959524,7.318957e+03,8.374953,...,2326.081648,2.021269,695.064178,0.249350,230.259498,0.804746,32.204247,0.019207,1290.799156,1.154542
std,15164.717720,51308.865791,47803.525596,3.955676e+05,20.250176,19.602745,13862.922934,13989.348252,4.004692e+05,8.214988,...,12159.928409,5.030497,3466.438554,0.693856,1387.887687,4.835109,435.841952,0.134887,5414.425184,1.864479
min,1001.000000,0.000000,0.000000,-1.000000e+02,0.000000,0.000000,0.000000,0.000000,-1.000000e+02,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18178.000000,1661.076794,1631.358726,-1.091343e+01,10.847099,10.927326,592.983070,608.235919,-1.232638e+01,3.442171,...,31.131894,0.197446,3.328667,0.027088,5.274117,0.026997,0.000000,0.000000,33.382261,0.221558
50%,29177.000000,4097.827186,4131.174278,-6.239322e-02,19.685956,19.200682,1500.780036,1636.858816,3.486028e+00,6.150772,...,136.051404,0.579669,16.473665,0.085588,20.850829,0.072033,1.000000,0.003242,132.098182,0.572597
75%,45082.000000,12954.123206,13119.029389,7.262080e+00,29.581054,28.862231,4591.067541,4859.016156,2.352342e+01,10.324935,...,874.891533,1.673132,130.916653,0.249311,101.353615,0.186646,7.107201,0.013054,658.924526,1.318699
max,56045.000000,886068.668386,740267.412661,2.208376e+07,100.000001,100.000001,292541.789025,259479.516033,2.234893e+07,72.274456,...,263440.500070,79.323968,76998.348562,25.087551,40350.800429,83.844210,21387.063061,4.469546,115781.021513,27.120652


In [94]:
# drop freatures that show abs number, keep percent pop features
access.drop(['State','County','LACCESS_POP10','LACCESS_POP15','LACCESS_LOWI10','LACCESS_LOWI15','LACCESS_CHILD10','LACCESS_CHILD_10_15','LACCESS_CHILD15','LACCESS_SENIORS10','LACCESS_SENIORS15','LACCESS_HHNV10','LACCESS_HHNV15','LACCESS_SNAP15','LACCESS_WHITE15','LACCESS_BLACK15','LACCESS_HISP15','LACCESS_NHASIAN15','LACCESS_NHNA15','LACCESS_NHPI15','LACCESS_MULTIR15'],axis=1,inplace=True)
access.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   FIPS                       3143 non-null   int64  
 1   PCH_LACCESS_POP_10_15      3117 non-null   float64
 2   PCT_LACCESS_POP10          3143 non-null   float64
 3   PCT_LACCESS_POP15          3124 non-null   float64
 4   PCH_LACCESS_LOWI_10_15     3115 non-null   float64
 5   PCT_LACCESS_LOWI10         3143 non-null   float64
 6   PCT_LACCESS_LOWI15         3123 non-null   float64
 7   PCH_LACCESS_HHNV_10_15     3129 non-null   float64
 8   PCT_LACCESS_HHNV10         3143 non-null   float64
 9   PCT_LACCESS_HHNV15         3140 non-null   float64
 10  PCT_LACCESS_SNAP15         3123 non-null   float64
 11  PCT_LACCESS_CHILD10        3143 non-null   float64
 12  PCT_LACCESS_CHILD15        3124 non-null   float64
 13  PCH_LACCESS_SENIORS_10_15  3117 non-null   float

In [95]:
# check correlation of access features
access.drop('FIPS',axis=1).corr()

,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,PCH_LACCESS_LOWI_10_15,PCT_LACCESS_LOWI10,PCT_LACCESS_LOWI15,PCH_LACCESS_HHNV_10_15,PCT_LACCESS_HHNV10,PCT_LACCESS_HHNV15,PCT_LACCESS_SNAP15,...,PCH_LACCESS_SENIORS_10_15,PCT_LACCESS_SENIORS10,PCT_LACCESS_SENIORS15,PCT_LACCESS_WHITE15,PCT_LACCESS_BLACK15,PCT_LACCESS_HISP15,PCT_LACCESS_NHASIAN15,PCT_LACCESS_NHNA15,PCT_LACCESS_NHPI15,PCT_LACCESS_MULTIR15
PCH_LACCESS_POP_10_15,1.000000,-0.021332,-0.011247,0.999987,-0.018666,-0.013945,-0.000698,-0.008273,-0.008749,-0.010878,...,0.999695,-0.016999,-0.003438,-0.008592,-0.008182,-0.006439,-0.005537,-0.002855,-0.002296,-0.009608
PCT_LACCESS_POP10,-0.021332,1.000000,0.853439,-0.021483,0.901991,0.743295,-0.013116,0.120353,0.052132,0.469834,...,-0.022092,0.919663,0.794667,0.824133,0.068671,0.339528,0.119603,0.217327,0.074109,0.458461
PCT_LACCESS_POP15,-0.011247,0.853439,1.000000,-0.011355,0.778241,0.902763,0.012386,0.126910,0.149968,0.636900,...,-0.011688,0.766717,0.908680,0.936824,0.140744,0.411640,0.161410,0.292461,0.115227,0.566459
PCH_LACCESS_LOWI_10_15,0.999987,-0.021483,-0.011355,1.000000,-0.018790,-0.014054,-0.000704,-0.008224,-0.008699,-0.010961,...,0.999808,-0.017100,-0.003511,-0.008687,-0.008223,-0.006468,-0.005565,-0.002855,-0.002291,-0.009706
PCT_LACCESS_LOWI10,-0.018666,0.901991,0.778241,-0.018790,1.000000,0.818331,-0.015105,0.278653,0.208247,0.625093,...,-0.019327,0.826792,0.722722,0.692157,0.160311,0.381987,0.042961,0.323858,0.052313,0.459066
PCT_LACCESS_LOWI15,-0.013945,0.743295,0.902763,-0.014054,0.818331,1.000000,0.014678,0.286250,0.325036,0.825737,...,-0.014286,0.652591,0.801443,0.769996,0.256670,0.464076,0.086723,0.416484,0.081516,0.563567
PCH_LACCESS_HHNV_10_15,-0.000698,-0.013116,0.012386,-0.000704,-0.015105,0.014678,1.000000,-0.036900,0.021169,0.002458,...,-0.000645,-0.004883,0.019901,0.015286,-0.008384,0.003583,0.000630,-0.000625,-0.000397,0.007174
PCT_LACCESS_HHNV10,-0.008273,0.120353,0.126910,-0.008224,0.278653,0.286250,-0.036900,1.000000,0.898309,0.469096,...,-0.007973,0.043476,0.042041,-0.057057,0.216522,-0.006224,-0.015140,0.525061,0.002074,0.033919
PCT_LACCESS_HHNV15,-0.008749,0.052132,0.149968,-0.008699,0.208247,0.325036,0.021169,0.898309,1.000000,0.529472,...,-0.008396,-0.011611,0.067063,-0.043798,0.247953,-0.005595,-0.013724,0.534175,0.010361,0.056996
PCT_LACCESS_SNAP15,-0.010878,0.469834,0.636900,-0.010961,0.625093,0.825737,0.002458,0.469096,0.529472,1.000000,...,-0.011110,0.350932,0.493608,0.417475,0.360452,0.372741,0.057594,0.580888,0.074041,0.437424


In [96]:
# import Store Availability data 
stores = pd.read_excel('data/FoodAvailabilityDataset.xls',sheet_name='STORES')
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   FIPS                 3143 non-null   int64  
 1   State                3143 non-null   object 
 2   County               3143 non-null   object 
 3   GROC11               3143 non-null   int64  
 4   GROC16               3143 non-null   int64  
 5   PCH_GROC_11_16       3122 non-null   float64
 6   GROCPTH11            3143 non-null   float64
 7   GROCPTH16            3143 non-null   float64
 8   PCH_GROCPTH_11_16    3127 non-null   float64
 9   SUPERC11             3143 non-null   int64  
 10  SUPERC16             3143 non-null   int64  
 11  PCH_SUPERC_11_16     3010 non-null   float64
 12  SUPERCPTH11          3143 non-null   float64
 13  SUPERCPTH16          3143 non-null   float64
 14  PCH_SUPERCPTH_11_16  3011 non-null   float64
 15  CONVS11              3143 non-null   i

In [97]:
# drop columns that contain absolute values. We can remove them since we have # per 1,000 pop columns that describe
# the same feature (e.g. "Grocery stores, 2007" vs "Grocery stores/1,000 pop, 2007")
stores.drop(['State','County','GROC11','GROC16','SUPERC11','SUPERC16','CONVS11','CONVS16','SPECS11','SPECS16','SNAPS12','SNAPS17','WICS11','WICS16'],axis=1,inplace=True)

In [98]:
# check feature correlation 
# FIPS (State and county Federal information processing standards) doesn't need to be included
stores.drop('FIPS',axis=1).corr()

,PCH_GROC_11_16,GROCPTH11,GROCPTH16,PCH_GROCPTH_11_16,PCH_SUPERC_11_16,SUPERCPTH11,SUPERCPTH16,PCH_SUPERCPTH_11_16,PCH_CONVS_11_16,CONVSPTH11,...,SPECSPTH16,PCH_SPECSPTH_11_16,PCH_SNAPS_12_17,SNAPSPTH12,SNAPSPTH17,PCH_SNAPSPTH_12_17,PCH_WICS_11_16,WICSPTH11,WICSPTH16,PCH_WICSPTH_11_16
PCH_GROC_11_16,1.000000,-0.173015,0.198001,0.988649,0.033164,0.015828,-0.011742,0.020932,-0.080294,0.001430,...,0.024917,0.002865,0.059529,-0.061356,-0.035361,0.046496,0.126547,-0.045220,-0.001011,0.109344
GROCPTH11,-0.173015,1.000000,0.851436,-0.148638,-0.095124,-0.202842,-0.210359,-0.078727,-0.011615,0.180818,...,0.074429,-0.022474,0.002999,0.338922,0.379782,0.035639,-0.069642,0.727945,0.730626,-0.032882
GROCPTH16,0.198001,0.851436,1.000000,0.226398,-0.086391,-0.205859,-0.220815,-0.070367,-0.050285,0.200799,...,0.082029,-0.019618,0.034099,0.325361,0.399666,0.068482,-0.021013,0.716705,0.747286,0.018195
PCH_GROCPTH_11_16,0.988649,-0.148638,0.226398,1.000000,0.007330,0.006460,-0.018979,0.009069,-0.085982,0.032913,...,0.012871,-0.008925,0.053295,-0.016345,0.021620,0.059550,0.102823,-0.009312,0.035459,0.111520
PCH_SUPERC_11_16,0.033164,-0.095124,-0.086391,0.007330,1.000000,-0.048708,0.107756,0.990389,-0.002952,-0.146118,...,0.043003,0.083062,-0.002382,-0.113273,-0.132647,-0.030591,0.075133,-0.119581,-0.104559,0.040998
SUPERCPTH11,0.015828,-0.202842,-0.205859,0.006460,-0.048708,1.000000,0.891392,-0.044249,-0.026785,-0.042360,...,0.030221,0.031616,-0.024261,-0.013477,-0.038034,-0.035690,-0.020352,-0.175485,-0.161622,-0.031478
SUPERCPTH16,-0.011742,-0.210359,-0.220815,-0.018979,0.107756,0.891392,1.000000,0.115589,-0.020677,-0.048126,...,0.031158,0.030899,-0.024031,-0.019048,-0.041669,-0.033703,0.001051,-0.184874,-0.165282,-0.007583
PCH_SUPERCPTH_11_16,0.020932,-0.078727,-0.070367,0.009069,0.990389,-0.044249,0.115589,1.000000,-0.006079,-0.121060,...,0.037929,0.068604,-0.008995,-0.079685,-0.092756,-0.022111,0.057027,-0.096609,-0.081993,0.041506
PCH_CONVS_11_16,-0.080294,-0.011615,-0.050285,-0.085982,-0.002952,-0.026785,-0.020677,-0.006079,1.000000,-0.264963,...,0.017999,-0.004973,0.039095,-0.050082,-0.048617,0.030760,0.050181,-0.028949,-0.008127,0.041646
CONVSPTH11,0.001430,0.180818,0.200799,0.032913,-0.146118,-0.042360,-0.048126,-0.121060,-0.264963,1.000000,...,-0.041284,-0.035991,-0.000119,0.366752,0.445710,0.034204,-0.052041,0.270373,0.261807,-0.002239


In [99]:
# import Restaurant Availability and Expenditures data
rstrnt = pd.read_excel('data/FoodAvailabilityDataset.xls',sheet_name='RESTAURANTS')
rstrnt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   FIPS              3143 non-null   int64  
 1   State             3143 non-null   object 
 2   County            3143 non-null   object 
 3   FFR11             3143 non-null   int64  
 4   FFR16             3143 non-null   int64  
 5   PCH_FFR_11_16     3086 non-null   float64
 6   FFRPTH11          3143 non-null   float64
 7   FFRPTH16          3143 non-null   float64
 8   PCH_FFRPTH_11_16  3091 non-null   float64
 9   FSR11             3143 non-null   int64  
 10  FSR16             3143 non-null   int64  
 11  PCH_FSR_11_16     3115 non-null   float64
 12  FSRPTH11          3143 non-null   float64
 13  FSRPTH16          3143 non-null   float64
 14  PCH_FSRPTH_11_16  3120 non-null   float64
 15  PC_FFRSALES07     3143 non-null   float64
 16  PC_FFRSALES12     3143 non-null   float64


In [100]:
# drop columns with data for 2012
# drop columns that contain absolute values and keep # per 1,000 pop columns
# drop columns that contain data for 2002: there are columns describing the same features for 2007
col_list = [col for col in rstrnt.columns if '12' in col]
rstrnt.drop(col_list, axis=1, inplace=True)
rstrnt.drop(['State','County','FFR11','FSR16','FSRPTH11','FSRPTH16','PC_FFRSALES07','PC_FSRSALES07'], axis=1, inplace=True)
rstrnt.drop('FIPS',axis=1).corr()

,FFR16,PCH_FFR_11_16,FFRPTH11,FFRPTH16,PCH_FFRPTH_11_16,FSR11,PCH_FSR_11_16,PCH_FSRPTH_11_16
FFR16,1.000000,0.030535,0.160841,0.177040,-0.000496,0.971886,0.043580,0.012668
PCH_FFR_11_16,0.030535,1.000000,-0.180252,0.237203,0.988431,0.025916,-0.065871,-0.078063
FFRPTH11,0.160841,-0.180252,1.000000,0.841992,-0.200886,0.168864,0.059112,0.036494
FFRPTH16,0.177040,0.237203,0.841992,1.000000,0.219590,0.181609,0.026241,0.006599
PCH_FFRPTH_11_16,-0.000496,0.988431,-0.200886,0.219590,1.000000,-0.004876,-0.083068,-0.076862
FSR11,0.971886,0.025916,0.168864,0.181609,-0.004876,1.000000,0.036937,0.006204
PCH_FSR_11_16,0.043580,-0.065871,0.059112,0.026241,-0.083068,0.036937,1.000000,0.989317
PCH_FSRPTH_11_16,0.012668,-0.078063,0.036494,0.006599,-0.076862,0.006204,0.989317,1.000000


In [101]:
# import Food Assistance data
assist = pd.read_excel('data/FoodAvailabilityDataset.xls',sheet_name='ASSISTANCE')
assist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 57 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   FIPS                      3143 non-null   int64  
 1   State                     3143 non-null   object 
 2   County                    3143 non-null   object 
 3   REDEMP_SNAPS12            2901 non-null   float64
 4   REDEMP_SNAPS17            2895 non-null   float64
 5   PCH_REDEMP_SNAPS_12_17    2844 non-null   float64
 6   PCT_SNAP12                3143 non-null   float64
 7   PCT_SNAP17                3143 non-null   float64
 8   PCH_SNAP_12_17            3143 non-null   float64
 9   PC_SNAPBEN12              3087 non-null   float64
 10  PC_SNAPBEN17              3087 non-null   float64
 11  PCH_PC_SNAPBEN_12_17      3087 non-null   float64
 12  SNAP_PART_RATE11          3143 non-null   float64
 13  SNAP_PART_RATE16          3143 non-null   float64
 14  SNAP_OAP

In [102]:
# # drop features that represent data for the years after 2010 (future data): 2012, 2014
# # when there is data describing the same feature for several years keep the most recent (i.e. closest to 2010)
# # caveat: for some features 2008 is the most recent year, but they have too many NULLs -> should be dropped
# Define the years to be removed.
years_to_remove = ['11', '12', '14', '16']

# Create a list of column names to be dropped that contain any of the years to be removed.
cols_to_drop = [col for col in assist.columns if any(year in col for year in years_to_remove)]

# Add 'State' and 'County' to the list of columns to be dropped.
cols_to_drop.extend(['State', 'County'])

# Drop the identified columns from the 'assist' DataFrame.
assist.drop(cols_to_drop, axis=1, inplace=True)

# Now, drop the 'FIPS' column and calculate the correlation matrix.
assist.drop('FIPS', axis=1).corr()


,REDEMP_SNAPS17,PCT_SNAP17,PC_SNAPBEN17,SNAP_OAPP09,SNAP_CAP09,SNAP_BBCE09,SNAP_REPORTSIMPLE09,PCT_NSLP17,PCT_FREE_LUNCH10,PCT_FREE_LUNCH15,PCT_REDUCED_LUNCH10,PCT_REDUCED_LUNCH15,PCT_SBP17,PCT_SFSP17,PCT_WIC17,PCT_CACFP17,FDPIR15,FOOD_BANKS18
REDEMP_SNAPS17,1.000000,0.281575,0.471347,-0.019807,0.137413,0.023940,-0.002029,-0.098509,0.280026,0.298918,-0.236139,-0.327248,0.130589,0.007520,0.150195,-0.015439,0.131358,0.290870
PCT_SNAP17,0.281575,1.000000,0.504826,-0.037756,0.261042,0.087280,0.196751,0.161268,0.493573,0.460426,-0.238626,-0.389067,0.670134,0.172954,0.524436,0.034214,0.019647,0.035285
PC_SNAPBEN17,0.471347,0.504826,1.000000,-0.035122,0.184497,0.003919,0.056278,0.031672,0.739970,0.759745,-0.235435,-0.467054,0.375680,0.092564,0.293947,-0.006988,0.108557,0.058986
SNAP_OAPP09,-0.019807,-0.037756,-0.035122,1.000000,0.197037,0.147432,0.060378,0.000535,-0.088344,-0.059811,-0.025609,-0.017511,-0.043457,-0.191111,-0.113923,0.005124,-0.038463,-0.003835
SNAP_CAP09,0.137413,0.261042,0.184497,0.197037,1.000000,0.048435,0.125077,-0.035472,0.183596,0.209039,-0.197707,-0.271777,0.242275,0.190829,0.311499,0.103314,-0.026382,0.024271
SNAP_BBCE09,0.023940,0.087280,0.003919,0.147432,0.048435,1.000000,-0.081810,-0.129784,-0.056856,0.013350,-0.057427,-0.043682,0.076126,-0.133514,0.169158,-0.016004,0.087386,0.084779
SNAP_REPORTSIMPLE09,-0.002029,0.196751,0.056278,0.060378,0.125077,-0.081810,1.000000,0.188241,0.038625,0.043481,-0.026655,-0.085092,0.103270,0.140869,-0.124540,-0.007861,-0.150087,-0.169470
PCT_NSLP17,-0.098509,0.161268,0.031672,0.000535,-0.035472,-0.129784,0.188241,1.000000,0.175694,0.108145,0.129958,0.065346,0.481556,-0.018053,0.381620,0.636045,-0.068400,-0.136345
PCT_FREE_LUNCH10,0.280026,0.493573,0.739970,-0.088344,0.183596,-0.056856,0.038625,0.175694,1.000000,0.860168,-0.092304,-0.264401,0.497822,0.064553,0.427672,0.098474,0.052361,-0.005009
PCT_FREE_LUNCH15,0.298918,0.460426,0.759745,-0.059811,0.209039,0.013350,0.043481,0.108145,0.860168,1.000000,-0.087396,-0.372892,0.500260,0.052603,0.366396,0.040863,0.069009,0.009265


In [103]:
# State Food Insecurity
insecur = pd.read_excel('data/FoodAvailabilityDataset.xls',sheet_name='INSECURITY')
insecur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   FIPS                3143 non-null   int64  
 1   State               3143 non-null   object 
 2   County              3143 non-null   object 
 3   FOODINSEC_12_14     3143 non-null   float64
 4   FOODINSEC_15_17     3143 non-null   float64
 5   CH_FOODINSEC_14_17  3143 non-null   float64
 6   VLFOODSEC_12_14     3143 non-null   float64
 7   VLFOODSEC_15_17     3143 non-null   float64
 8   CH_VLFOODSEC_14_17  3143 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 221.1+ KB


In [104]:
# keep features from the most recent year when there is data for more than one year available
insecur=insecur[['FIPS','FOODINSEC_15_17','CH_FOODINSEC_14_17','VLFOODSEC_15_17','CH_VLFOODSEC_14_17']]
insecur.drop('FIPS',axis=1).corr()

,FOODINSEC_15_17,CH_FOODINSEC_14_17,VLFOODSEC_15_17,CH_VLFOODSEC_14_17
FOODINSEC_15_17,1.000000,0.126667,0.928356,0.420418
CH_FOODINSEC_14_17,0.126667,1.000000,0.071294,0.658935
VLFOODSEC_15_17,0.928356,0.071294,1.000000,0.476470
CH_VLFOODSEC_14_17,0.420418,0.658935,0.476470,1.000000


In [105]:
# Food Prices and Taxes
prtax = pd.read_excel('data/FoodAvailabilityDataset.xls',sheet_name='TAXES')
prtax.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   FIPS               3143 non-null   int64  
 1   State              3143 non-null   object 
 2   County             3143 non-null   object 
 3   SODATAX_STORES14   3143 non-null   float64
 4   SODATAX_VENDM14    3143 non-null   float64
 5   CHIPSTAX_STORES14  3143 non-null   float64
 6   CHIPSTAX_VENDM14   3143 non-null   float64
 7   FOOD_TAX14         3143 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 196.6+ KB


In [106]:
numeric_df = prtax.apply(pd.to_numeric, errors='coerce')

# Drop the 'FIPS' column
numeric_df = numeric_df.drop('FIPS', axis=1)

# Calculate the correlation matrix
correlation_matrix = numeric_df.corr()

# Print the correlation matrix
print(correlation_matrix)

                   State  County  SODATAX_STORES14  SODATAX_VENDM14  \
State                NaN     NaN               NaN              NaN   
County               NaN     NaN               NaN              NaN   
SODATAX_STORES14     NaN     NaN          1.000000         0.775872   
SODATAX_VENDM14      NaN     NaN          0.775872         1.000000   
CHIPSTAX_STORES14    NaN     NaN          0.205313         0.162123   
CHIPSTAX_VENDM14     NaN     NaN          0.457167         0.704771   
FOOD_TAX14           NaN     NaN          0.205313         0.162123   

                   CHIPSTAX_STORES14  CHIPSTAX_VENDM14  FOOD_TAX14  
State                            NaN               NaN         NaN  
County                           NaN               NaN         NaN  
SODATAX_STORES14            0.205313          0.457167    0.205313  
SODATAX_VENDM14             0.162123          0.704771    0.162123  
CHIPSTAX_STORES14           1.000000          0.266035    1.000000  
CHIPSTAX_VENDM14 

In [107]:
# Local Foods
local = pd.read_excel('data/FoodAvailabilityDataset.xls',sheet_name='LOCAL')
local.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3144 entries, 0 to 3143
Data columns (total 100 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   FIPS                         3143 non-null   float64
 1   State                        3143 non-null   object 
 2   County                       3143 non-null   object 
 3   DIRSALES_FARMS07             3080 non-null   float64
 4   DIRSALES_FARMS12             3080 non-null   float64
 5   PCH_DIRSALES_FARMS_07_12     3019 non-null   float64
 6   PCT_LOCLFARM07               3074 non-null   float64
 7   PCT_LOCLFARM12               3075 non-null   float64
 8   PCT_LOCLSALE07               2797 non-null   float64
 9   PCT_LOCLSALE12               2856 non-null   float64
 10  DIRSALES07                   2855 non-null   float64
 11  DIRSALES12                   2901 non-null   float64
 12  PCH_DIRSALES_07_12           2684 non-null   float64
 13  PC_DIRSALES07    

In [108]:


# Drop columns with data for the year 2013
cols = [col for col in local.columns if '13' in col]
local.drop(cols, axis=1, inplace=True)

# Drop absolute features represented in count/acres when there is a feature represented in count/acres per 1,000 pop
# Drop 2012 feature as well (if there are any, the list doesn't contain '12' but it's mentioned in the comment)
abs_feat = ['DIRSALES_FARMS07', 'VEG_ACRES07', 'FRESHVEG_ACRES07', 'ORCHARD_ACRES07', 'BERRY_ACRES07', 'GHVEG_SQFT07']
local.drop(abs_feat, axis=1, inplace=True)

# Attempt to convert all columns to numeric, coercing errors to NaN
local_numeric = local.apply(pd.to_numeric, errors='coerce')

# Drop the 'FIPS' column from this new numeric DataFrame
local_numeric = local_numeric.drop('FIPS', axis=1)

# Compute the correlation matrix for the numeric columns
correlation_matrix = local_numeric.corr()

# Print the correlation matrix
print(correlation_matrix)


                          State  County  DIRSALES_FARMS12  \
State                       NaN     NaN               NaN   
County                      NaN     NaN               NaN   
DIRSALES_FARMS12            NaN     NaN          1.000000   
PCH_DIRSALES_FARMS_07_12    NaN     NaN          0.006103   
PCT_LOCLFARM07              NaN     NaN          0.456574   
...                         ...     ...               ...   
PCH_AGRITRSM_OPS_07_12      NaN     NaN          0.085476   
AGRITRSM_RCT07              NaN     NaN          0.239787   
AGRITRSM_RCT12              NaN     NaN          0.267738   
PCH_AGRITRSM_RCT_07_12      NaN     NaN          0.035708   
FARM_TO_SCHOOL15            NaN     NaN          0.315455   

                          PCH_DIRSALES_FARMS_07_12  PCT_LOCLFARM07  \
State                                          NaN             NaN   
County                                         NaN             NaN   
DIRSALES_FARMS12                          0.006103       

In [109]:
# drop features with over 255 NULLs
local.dropna(thresh=3000,axis=1,inplace=True)
local.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3144 entries, 0 to 3143
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   FIPS                      3143 non-null   float64
 1   State                     3143 non-null   object 
 2   County                    3143 non-null   object 
 3   DIRSALES_FARMS12          3080 non-null   float64
 4   PCH_DIRSALES_FARMS_07_12  3019 non-null   float64
 5   PCT_LOCLFARM07            3074 non-null   float64
 6   PCT_LOCLFARM12            3075 non-null   float64
 7   FMRKT18                   3142 non-null   float64
 8   FMRKTPTH18                3140 non-null   float64
 9   FMRKT_SNAP18              3142 non-null   float64
 10  PCT_FMRKT_SNAP18          3142 non-null   float64
 11  FMRKT_WIC18               3142 non-null   float64
 12  PCT_FMRKT_WIC18           3142 non-null   float64
 13  FMRKT_WICCASH18           3142 non-null   float64
 14  PCT_FMRK

In [110]:
# Socioeconomic Characteristics
secon = pd.read_excel('data/FoodAvailabilityDataset.xls',sheet_name='SOCIOECONOMIC')
secon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FIPS             3143 non-null   int64  
 1   State            3143 non-null   object 
 2   County           3143 non-null   object 
 3   PCT_NHWHITE10    3143 non-null   float64
 4   PCT_NHBLACK10    3143 non-null   float64
 5   PCT_HISP10       3143 non-null   float64
 6   PCT_NHASIAN10    3143 non-null   float64
 7   PCT_NHNA10       3143 non-null   float64
 8   PCT_NHPI10       3143 non-null   float64
 9   PCT_65OLDER10    3143 non-null   float64
 10  PCT_18YOUNGER10  3143 non-null   float64
 11  MEDHHINC15       3139 non-null   float64
 12  POVRATE15        3139 non-null   float64
 13  PERPOV10         3143 non-null   int64  
 14  CHILDPOVRATE15   3139 non-null   float64
 15  PERCHLDPOV10     3143 non-null   int64  
 16  METRO13          3143 non-null   int64  
 17  POPLOSS10     

In [111]:
# create a list of dataframes
dfs = [access,stores,rstrnt,assist,insecur,prtax,local,health,secon,obesity]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   FIPS                3143 non-null   int64  
 1   State               3143 non-null   object 
 2   County              3143 non-null   object 
 3   PCT_OBESE_ADULTS17  3143 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 98.3+ KB


In [114]:
# Merge all dataframes on their 'FIPS' column
data = reduce(lambda left, right: pd.merge(left, right, on='FIPS', how='outer', suffixes=('', '_dup')), dfs)

In [115]:
data.index=data.FIPS

In [116]:
data.drop('FIPS',axis=1,inplace=True)
data.head()

,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,PCH_LACCESS_LOWI_10_15,PCT_LACCESS_LOWI10,PCT_LACCESS_LOWI15,PCH_LACCESS_HHNV_10_15,PCT_LACCESS_HHNV10,PCT_LACCESS_HHNV15,PCT_LACCESS_SNAP15,...,MEDHHINC15,POVRATE15,PERPOV10,CHILDPOVRATE15,PERCHLDPOV10,METRO13,POPLOSS10,State_dup,County_dup,PCT_OBESE_ADULTS17
FIPS,,,,,,,,,,,,,,,,,,,,,
1001.0,-5.056026,33.769657,32.062255,22.439248,9.793530,11.991125,2.025888,3.284786,3.351332,4.608749,...,56580.0,12.7,0.0,18.8,0.0,1.0,0.0,AL,Autauga,36.3
1003.0,-13.204891,19.318473,16.767489,-0.656270,5.460261,5.424427,-11.300383,2.147827,1.905114,1.298900,...,52387.0,12.9,0.0,19.6,0.0,1.0,0.0,AL,Baldwin,36.3
1005.0,6.067799,20.840972,22.105560,-5.959985,11.420316,10.739667,4.678792,4.135869,4.329378,4.303147,...,31433.0,32.0,1.0,45.2,1.0,0.0,0.0,AL,Barbour,36.3
1007.0,-7.224696,4.559753,4.230324,21.307144,2.144661,2.601627,-18.422377,3.458580,2.821427,0.676710,...,40767.0,22.2,0.0,29.3,1.0,1.0,0.0,AL,Bibb,36.3
1009.0,140.568857,2.700840,6.497380,171.081177,1.062468,2.880150,2.050367,3.269380,3.336414,0.812727,...,50487.0,14.7,0.0,22.2,0.0,1.0,0.0,AL,Blount,36.3


In [117]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3144 entries, 1001.0 to nan
Columns: 147 entries, PCH_LACCESS_POP_10_15 to PCT_OBESE_ADULTS17
dtypes: float64(139), object(8)
memory usage: 3.6+ MB


In [118]:
#remove duplicate rows
data.drop_duplicates(inplace=True)
len(data)

3144

In [119]:
data.to_csv('food_atlas_combined.csv')